In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import os
import re
import pandas as pd

In [ ]:
url_blaser = 'https://www.blasercafe.ch/shop/category/8'
url_adrianos = 'https://adrianos.ch/shop/kaffee/gemahlen'

driver = webdriver.Chrome()

driver.get(url_blaser)

In [19]:


soup = bs(driver.page_source, 'html.parser')
with open('soup_blaser.txt', 'w', encoding='utf-8') as file:
    file.write(str(soup))

content = soup.find_all("div", class_=re.compile("views-row"))

products_blaser = []

for c in content: 
    try:
        product = c.find("div", class_=re.compile("product-teaser-content"))
        

        a = product.find("a")

        name = a.find("span").text.strip()

        product_wrapper = c.find("div", class_=re.compile("product-teaser-content-wrapper"))

        price = product_wrapper.find("span").text.strip()
        
        #remove text
        price_transformed = price[-(len(price)-7):]

        products_blaser.append({'name':name, 'price': price_transformed})
        
    
    except Exception as e:
        #this row has no products, so ignore
        continue

[{'name': 'Terra Vita Bio Fairtrade Bohnenkaffee (bisher Pura Vida)', 'price': '6.90'}]
[{'name': 'Terra Vita Bio Fairtrade Bohnenkaffee (bisher Pura Vida)', 'price': '6.90'}, {'name': 'Rösterei Spezial Maya Bohnenkaffee', 'price': '7.90'}]
[{'name': 'Terra Vita Bio Fairtrade Bohnenkaffee (bisher Pura Vida)', 'price': '6.90'}, {'name': 'Rösterei Spezial Maya Bohnenkaffee', 'price': '7.90'}, {'name': 'Coffee for Champions - Bohnenkaffee', 'price': '8.50'}]
[{'name': 'Terra Vita Bio Fairtrade Bohnenkaffee (bisher Pura Vida)', 'price': '6.90'}, {'name': 'Rösterei Spezial Maya Bohnenkaffee', 'price': '7.90'}, {'name': 'Coffee for Champions - Bohnenkaffee', 'price': '8.50'}, {'name': 'Marrone Bohnenkaffee', 'price': '6.10'}]
[{'name': 'Terra Vita Bio Fairtrade Bohnenkaffee (bisher Pura Vida)', 'price': '6.90'}, {'name': 'Rösterei Spezial Maya Bohnenkaffee', 'price': '7.90'}, {'name': 'Coffee for Champions - Bohnenkaffee', 'price': '8.50'}, {'name': 'Marrone Bohnenkaffee', 'price': '6.10'}, 

In [23]:
#adrianos
url_adrianos = 'https://adrianos.ch/shop/kaffee/gemahlen'

driver = webdriver.Chrome()

driver.get(url_adrianos)

In [29]:


soup = bs(driver.page_source, 'html.parser')
with open('soup_adrianos.txt', 'w', encoding='utf-8') as file:
    file.write(str(soup))

content = soup.find_all("article", class_=re.compile("product product_preview"))

products_adrianos = []

for c in content: 
    try:
        product = c.find("div", class_=re.compile("product__caption product__caption_preview"))

        name = product.find("h2").text.strip()

        price = product.find("span").text.strip()
        
        #remove text
        price_transformed = price[-(len(price)-3):]

        products_adrianos.append({'name':name, 'price': price_transformed})
        
        print(products_adrianos)
    
    except Exception as e:
        #this row has no products, so ignore
        continue

[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 15.00'}]
[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 15.00'}, {'name': 'Festive Blend 2023 Kaffee gemahlen', 'price': ' 12.50'}]
[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 15.00'}, {'name': 'Festive Blend 2023 Kaffee gemahlen', 'price': ' 12.50'}, {'name': 'Juanita Filterkaffee gemahlen', 'price': ' 13.00'}]
[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 15.00'}, {'name': 'Festive Blend 2023 Kaffee gemahlen', 'price': ' 12.50'}, {'name': 'Juanita Filterkaffee gemahlen', 'price': ' 13.00'}, {'name': 'Don Victor Kaffee gemahlen', 'price': ' 14.00'}]
[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 15.00'}, {'name': 'Festive Blend 2023 Kaffee gemahlen', 'price': ' 12.50'}, {'name': 'Juanita Filterkaffee gemahlen', 'price': ' 13.00'}, {'name': 'Don Victor Kaffee gemahlen', 'price': ' 14.00'}, {'name': 'Lila Sorto Filterkaffee gemahlen', 'price': ' 14.00'}]
[{'name': 'Tueste Filterkaffee gemahlen', 'price': ' 